# Triggered Request from GCP Cloud Functions

**Note**: This notebook runs on Python 3.9 and uses UbiOps CLient Library 3.15.0.

This notebook will cover the steps required to create a dummy deployment and/or pipeline. The first part of this notebook will tackle the deployment creation process; afterwards, in the final part, a dummy pipeline will be created, making use of the deployment created in the previous step. 

We recommend to run the cells step by step, as some cells can take a few minutes to finish. You can run everything in one go as well and it will work, just allow a few minutes for building the deployments.

## Establishing a connection with your UbiOps environment
Add your API token and then provide a project name. Afterwards we initialize the client library. This way we can deploy the deployment and/or pipeline to your environment.

In [2]:
API_TOKEN = '<INSERT API_TOKEN WITH PROJECT EDITOR RIGHTS>' # Make sure this is in the format "Token token-code"
PROJECT_NAME= '<INSERT PROJECT NAME IN YOUR ACCOUNT>'

# Import all necessary libraries
import requests
import ubiops

client = ubiops.ApiClient(ubiops.Configuration(api_key={'Authorization': API_TOKEN}, 
                                               host='https://api.ubiops.com/v2.1'))
api = ubiops.CoreApi(client)

## Create the dummy deployment

This step will create and deploy a deployment to your UbiOps environment. The deployment's functionality will consist of just passing the input as an output (hence it being called a dummy deployment).'

First, change the *DEPLOYMENT_NAME* and *DEPLOYMENT_VERSION* variables in the next cell to a deployment and version name of your choice. Then, simply run the following cell to create the dummy deployment.

In [4]:
DEPLOYMENT_NAME='example-deployment'
DEPLOYMENT_VERSION='v1'

deployment_template = ubiops.DeploymentCreate(
    name=DEPLOYMENT_NAME,
    description='Example deployment',
    input_type='structured',
    output_type='structured',
    input_fields=[
        {'name':'input', 'data_type':'string'}
    ],
    output_fields=[
        {'name':'output', 'data_type':'string'}
    ],
    labels={'demo': 'gcp-functions-tutorial'}
)

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template
)

# Create the version
version_template = ubiops.DeploymentVersionCreate(
    version=DEPLOYMENT_VERSION,
    environment='python3-9',
    instance_type='512mb',
    minimum_instances=0,
    maximum_instances=1,
    maximum_idle_time=1800, # = 30 minutes
    request_retention_mode='full', # input/output of requests will be stored
    request_retention_time=3600 # requests will be stored for 1 hour
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)

# Download the deployment package
dummy_package = requests.get('https://storage.googleapis.com/ubiops/data/Integration%20with%20cloud%20provider%20tools/google-cloud-functions-triggered-request/dummy_package.zip')

with open("dummy_package.zip", "wb") as f:
    f.write(dummy_package.content)

# Upload the zipped deployment package
file_upload_result =api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='dummy_package.zip'
)

# Deployment created and deployed!

Now that the deployment has been successfully created and deployed, we can move to the next step. 

The next step will continuously check and monitor the UbiOps environment in order to make sure that the created deployment has passed the building stage.

In [ ]:
ubiops.utils.wait_for_deployment_version(
    client=api.api_client,
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    revision_id=file_upload_result.revision
)

# Deployment passed the building stage

If the output of the previous cell is "available", the deployment has been successfully created, deployed and build. If you need to also create a dummy pipeline on top of the deployment, you can continue to the next step.

In the next step, we will create a simple pipeline which uses the deployment that we have created in the previous cells of this notebook. This pipeline will consist of only one deployment (the one previously created) and it will not have any real functionality.

First, give the dummy pipeline a name of your choice by changing the *PIPELINE_NAME* variable in the next cell. Then, simply run the cell to create the pipeline. 

In [ ]:
PIPELINE_NAME = "example-pipeline"

pipeline_template = ubiops.PipelineCreate(
    name=PIPELINE_NAME,
    description="An example pipeline",
    input_type='structured',
    input_fields=[
        {'name':'input', 'data_type':'string'}
    ],
    output_type='structured',
    output_fields=[
        {'name':'output', 'data_type':'string'}
    ],
    labels={'demo': 'gcp-functions-tutorial'}
)

api.pipelines_create(
    project_name=PROJECT_NAME,
    data=pipeline_template
)

PIPELINE_VERSION = DEPLOYMENT_VERSION

pipeline_template = ubiops.PipelineVersionCreate(
    version=PIPELINE_VERSION,
    request_retention_mode='full',
    objects=[
        # example deployment
        {
            'name': DEPLOYMENT_NAME,
            'reference_name': DEPLOYMENT_NAME,
            'version': DEPLOYMENT_VERSION
        }
    ],
    attachments=[
        # start --> example deployment
        {
            'destination_name': DEPLOYMENT_NAME,
            'sources': [{
                'source_name': 'pipeline_start',
                'mapping': [{
                    "source_field_name": 'input',
                    'destination_field_name': 'input'
                }]
            }]
        },
        # example deployment --> pipeline end
        {
            'destination_name': 'pipeline_end',
            'sources': [{
                'source_name': DEPLOYMENT_NAME,
                'mapping': [{
                    "source_field_name": 'output',
                    'destination_field_name': 'output'
                }]
            }]
        }
    ]
)

api.pipeline_versions_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=pipeline_template)

**IMPORTANT**: If you get an error like: "error":"Version is not available: The version is currently in the building stage"
Your deployment is not yet available and still building. 
Check in the UI if your deployment is ready and then rerun the block above.

# Pipeline successfully created!